In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
NOTA: Para este ejemplo asumiremos que los datos de entrenamiento son sobre los que trabajaremos

In [2]:
Y = np.array([[3], [1], [8], [3], [5]])
Y

array([[3],
       [1],
       [8],
       [3],
       [5]])

In [3]:
X = np.array([[1,3,5], [1,1,4], [1,5,6], [1,2,4], [1,4,6]])
X

array([[1, 3, 5],
       [1, 1, 4],
       [1, 5, 6],
       [1, 2, 4],
       [1, 4, 6]])

In [4]:
XT_X = np.matmul(np.matrix.transpose(X), X)
XT_X

array([[  5,  15,  25],
       [ 15,  55,  81],
       [ 25,  81, 129]])

In [5]:
XT_X_inv = np.linalg.inv(XT_X)
XT_X_inv

array([[26.7,  4.5, -8. ],
       [ 4.5,  1. , -1.5],
       [-8. , -1.5,  2.5]])

In [6]:
XT_Y = np.matmul(np.matrix.transpose(X), Y)
XT_Y

array([[ 20],
       [ 76],
       [109]])

In [7]:
betas = np.matmul(XT_X_inv, XT_Y)
betas

array([[ 4. ],
       [ 2.5],
       [-1.5]])

# Cálculo de los pronósticos para Y de acuerdo a los coeficientes de regresión.

In [8]:
Y_pred = np.matmul(X, betas)
Y_pred

array([[4. ],
       [0.5],
       [7.5],
       [3. ],
       [5. ]])

#  Cálculo de residuales.

In [11]:
Resid = Y - Y_pred
Resid

array([[-1.00000000e+00],
       [ 5.00000000e-01],
       [ 5.00000000e-01],
       [-3.73034936e-14],
       [-6.66133815e-14]])

#  Cálculo de la suma de residuales al cuadrado.

In [13]:
RSS = float(np.matmul(np.matrix.transpose(Resid), Resid))
RSS

1.5000000000000036

# Cálculo de la suma total de cuadrados.

In [15]:
TSS = float(np.matmul(np.matrix.transpose(Y), Y) - len(Y)*(Y.mean()**2))
TSS

28.0

# Cálculo del coeficiente de determinación.

In [16]:
R_cuad = float(1 - RSS/TSS)
R_cuad

0.9464285714285713

#  Cálculo del coeficiente de determinación ajustado.

In [18]:
RSqAj = float(1 - (RSS / (X.shape[0] - X.shape[1])) / (TSS / (X.shape[0] - 1)))
RSqAj

0.8928571428571426

# Cálculo de la varianza del error de regresión

In [19]:
s_cuad = RSS / (len(Y) - X.shape[1])
s_cuad

0.7500000000000018

# Desviación estándar del error de regresión.

In [20]:
import math

s = math.sqrt(s_cuad)
s

0.8660254037844397

#  Cálculo de las t´s estadísticas para cada coeficiente de regresión.

In [21]:
result_t = []
for i in range(0, X.shape[1]):
    t = float(betas[i] / (s * math.sqrt(XT_X_inv[i][i])))
    result_t.append(t)
result_t    

[0.8938686975386521, 2.88675134594813, -1.0954451150103248]

#  Criterio 1:

# Obtener valor crítico de la t de Student de tablas.

In [23]:
import scipy.stats

grados_libertad = len(Y) - X.shape[1]

# La t critica se obtendrá a un nivel de confianza del 95% (Alfa = 5%)
t_critico = abs(scipy.stats.t.ppf(q = 0.025, df = grados_libertad))
t_critico

4.3026527299112765

In [24]:
for i in range(0, X.shape[1]):
    if (abs(result_t[i]) > t_critico):
        print("Beta", i, "es significativa")       #-------> Aquí se rechaza H0
    else:
        print("Beta", i, "NO es significativa")       #-------> Aquí NO se rechaza H0   
     

Beta 0 NO es significativa
Beta 1 NO es significativa
Beta 2 NO es significativa


#  Criterio 2:

#  Cálculo de valores p

In [25]:
for i in range(0, X.shape[1]):
    print("Valor p de Beta", i, ":", scipy.stats.t.sf(abs(result_t[i]), df = grados_libertad) * 2)

Valor p de Beta 0 : 0.4657159826085351
Valor p de Beta 1 : 0.10197348986612542
Valor p de Beta 2 : 0.3876275643042082


In [ ]:
Si manejamos un nivel alfa del 5%, en ninguno de los casos el valor p es menor al 5%, por lo que no podemos rechazar H0

# Criterio 3:

#  Cálculo de intervalos de confianza del 95% para el verdadero valor del coeficiente de cada Beta.

In [27]:
for i in range(0, X.shape[1]):
    print("El valor de Beta", i, "se encuentra entre: ", float(betas[i]) - t_critico * s * math.sqrt(XT_X_inv[i][i]),
         "y", float((betas[i]) + t_critico * s * math.sqrt(XT_X_inv[i][i])))

El valor de Beta 0 se encuentra entre:  -15.25407049943246 y 23.254070499432444
El valor de Beta 1 se encuentra entre:  -1.2262065677656317 y 6.226206567765644
El valor de Beta 2 se encuentra entre:  -7.391649893208971 y 4.391649893208987


# Conclusión:

"Ninguna de las variables regresoras (independientes) es significativamente distinta de cero"

# Comparación de resultados contra reporte automatizado.

In [29]:
import statsmodels.api as sm

regressor = sm.OLS(Y, X).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     17.67
Date:                Sun, 02 Jun 2024   Prob (F-statistic):             0.0536
Time:                        18:52:17   Log-Likelihood:                -4.0848
No. Observations:                   5   AIC:                             14.17
Df Residuals:                       2   BIC:                             13.00
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0000      4.475      0.894      0.4

C:\Users\gilgu\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [30]:
data = pd.DataFrame(X)
data2 = data.iloc[:,1:3]
data2.corr()

,1,2
1,1.000000,0.948683
2,0.948683,1.000000


In [31]:
X_Nueva = np.delete(X, 2, 1)
X_Nueva

array([[1, 3],
       [1, 1],
       [1, 5],
       [1, 2],
       [1, 4]])

In [32]:
regressor = sm.OLS(Y, X_Nueva).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     32.00
Date:                Sun, 02 Jun 2024   Prob (F-statistic):             0.0109
Time:                        19:13:54   Log-Likelihood:                -5.2598
No. Observations:                   5   AIC:                             14.52
Df Residuals:                       3   BIC:                             13.74
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8000      0.938     -0.853      0.4

C:\Users\gilgu\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
